In [ ]:
import pandas as pd

researcher_nodes = pd.read_excel("nodes_undirected.xlsx")
coauthorship_edges = pd.read_excel("edges_undirected.xlsx")

# Parse the list of country codes associated with each researcher
# Splits semicolon-separated country codes into lists
researcher_nodes["country_list"] = researcher_nodes["Country Codes"].fillna("").apply(
    lambda x: [c.strip() for c in x.split(";") if c.strip()]
)

# Create a dictionary mapping each researcher (node) to their list of countries
author_to_countries = dict(zip(researcher_nodes["Id"], researcher_nodes["country_list"]))

# Define a function that takes an edge (Source–Target) and returns all valid
# cross-country collaboration pairs (e.g., ('US', 'GB')) for that co-authorship
def extract_country_pairs(edge_row):
    src_countries = author_to_countries.get(edge_row["Source"], [])
    tgt_countries = author_to_countries.get(edge_row["Target"], [])
    return [tuple(sorted((src, tgt))) for src in src_countries for tgt in tgt_countries if src != tgt]

# Apply the extraction function to all edges
coauthorship_edges["country_pairs"] = coauthorship_edges.apply(extract_country_pairs, axis=1)

# Explode the resulting list of country pairs into separate rows
country_collab_rows = coauthorship_edges.explode("country_pairs").dropna()

# Split each country pair tuple into two separate columns
country_collab_rows[["Country_1", "Country_2"]] = pd.DataFrame(
    country_collab_rows["country_pairs"].tolist(), index=country_collab_rows.index
)

# Group by each country pair and sum their edge weights (number of shared co-authorships)
aggregated_country_edges = (
    country_collab_rows.groupby(["Country_1", "Country_2"])["Weight"].sum().reset_index()
)

aggregated_country_edges.to_csv("country_collaborations.csv", index=False)
